In [3]:
import os
os.chdir("C:/Matott/Teaching/EAS503/eas503_notes/ml101/docs/assets")


In [4]:
datafile='countywide_covid_19_data.csv'
metadatafile='countywide_covid_19_metadata.csv'

In [6]:
#
# Part 1 - load the data and required modules
#
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve
import pandas as pd
import numpy as np
from six import StringIO
from matplotlib import pyplot as plt
import os

In [7]:
# adjust Pandas printing behavior
pd.set_option("display.max_rows", None)

In [8]:
# read data from file
df = pd.read_csv(datafile)

In [10]:
# inspect the dataframe, verify it looks ok
print("")
print("First 10 features:")
print(df.iloc[0:9,1:8])  # state + county + first 10 features


First 10 features:
  State           County  X01     X02  X03  X04  X05
0    AL   Autauga County    0   55869 -0.3    2    2
1    AL   Baldwin County    5  223234  0.4    3    2
2    AL   Barbour County    3   24686  0.5    6    6
3    AL      Bibb County    0   22394  0.4    1    1
4    AL    Blount County    0   57826  0.1    1    1
5    AL   Bullock County    3   10101 -0.1    6    6
6    AL    Butler County    0   19448  0.9    6    6
7    AL   Calhoun County    4  113605  0.1    3    2
8    AL  Chambers County    0   33254  0.2    6    5


In [11]:
print("")
print("Labels:")
print(df.iloc[0:9,[1,2,-5,-4,-3,-2,-1]]) # state + county + labels


Labels:
  State           County    Y01  Y02   Y03  Y04  Y05
0    AL   Autauga County   5407   62  1.15    4    3
1    AL   Baldwin County  17333  189  1.09    4    3
2    AL   Barbour County   1834   38  2.07    5    7
3    AL      Bibb County   2229   48  2.15    5    8
4    AL    Blount County   5485   86  1.57    4    5
5    AL   Bullock County   1046   28  2.68    5    9
6    AL    Butler County   1748   60  3.43    5   10
7    AL   Calhoun County  11555  206  1.78    4    6
8    AL  Chambers County   3054   70  2.29    5    8


In [13]:
df.tail(n=10)

,FIPS,State,County,X01,X02,X03,X04,X05,X06,X07,...,X50,X51,X52,X53,X54,Y01,Y02,Y03,Y04,Y05
3130,56027,WY,Niobrara County,0,2356,0.0,9,12,11.9,31.3,...,9.17,49.4,1.0,95.5,141.5,145,2,1.38,4,5
3131,56029,WY,Park County,5,29194,-0.3,7,11,5.2,26.7,...,18.01,42.3,2.0,94.8,144.7,2518,18,0.71,3,2
3132,56031,WY,Platte County,0,8393,-1.2,7,11,6.6,37.0,...,7.61,50.7,1.0,144.7,144.1,543,10,1.84,4,7
3133,56033,WY,Sheridan County,0,30485,0.5,7,8,4.8,24.3,...,12.21,46.7,1.0,69.9,168.5,2838,18,0.63,3,1
3134,56035,WY,Sublette County,2,9831,-0.8,9,10,3.7,35.5,...,12.49,39.0,0.0,70.0,131.5,641,7,1.09,4,3
3135,56037,WY,Sweetwater County,2,42343,0.5,5,8,9.0,33.3,...,5.31,45.7,2.0,126.3,180.5,3418,30,0.88,3,2
3136,56039,WY,Teton County,5,23464,3.4,7,8,5.6,13.2,...,27.17,38.4,1.0,28.8,86.0,2981,5,0.17,2,1
3137,56041,WY,Uinta County,2,20226,0.0,7,8,7.2,40.1,...,6.00,43.6,1.0,97.5,172.5,1913,12,0.63,3,1
3138,56043,WY,Washakie County,0,7805,-0.6,7,11,10.4,30.3,...,6.88,48.5,1.0,127.8,158.5,860,25,2.91,5,9
3139,56045,WY,Weston County,2,6927,2.6,7,9,7.8,35.8,...,8.93,48.2,1.0,134.9,154.7,610,4,0.66,3,1


In [14]:
# inspect categorical variables
print("")
print("Distribution of PSI labels:")
print("Label Count")
print("----- -----")
PSI = df["Y04"].astype('category')
print(PSI.value_counts().sort_index())



Distribution of PSI labels:
Label Count
----- -----
1      79
2     123
3     500
4    1370
5     950
6     105
7       9
8       4
Name: Y04, dtype: int64


In [15]:
print("")
print("Distribution of DRP labels:")
print("Label Count")
print("----- -----")
DRP = df["Y05"].astype('category')
print(DRP.value_counts().sort_index())


Distribution of DRP labels:
Label Count
----- -----
1     314
2     314
3     314
4     314
5     314
6     314
7     314
8     314
9     314
10    314
Name: Y05, dtype: int64


In [16]:
#
# Part 2 - select features and label
#

# extract features and labels
features =  [ "X{:02d}".format(i) for i in range(1,11)]
label    = "Y05"
X = df[features]
Y = df[label]

In [17]:
X.head(n=10)

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10
0,0,55869,-0.3,2,2,11.3,32.6,28.4,27.7,13.8
1,5,223234,0.4,3,2,9.7,27.6,31.3,31.3,9.8
2,3,24686,0.5,6,6,27.0,35.7,25.1,12.2,30.9
3,0,22394,0.4,1,1,16.8,47.3,24.4,11.5,21.8
4,0,57826,0.1,1,1,19.8,34.0,33.5,12.6,13.2
5,3,10101,-0.1,6,6,24.8,39.7,22.3,13.3,42.5
6,0,19448,0.9,6,6,15.4,43.9,24.6,16.1,24.5
7,4,113605,0.1,3,2,15.9,32.4,33.7,18.0,19.5
8,0,33254,0.2,6,5,18.6,38.4,29.7,13.2,18.7
9,0,26196,-0.1,6,6,19.8,38.3,29.0,12.9,16.3


In [18]:
Y.head(n=10)

0     3
1     3
2     7
3     8
4     5
5     9
6    10
7     6
8     8
9     7
Name: Y05, dtype: int64

In [19]:
mdf = pd.read_csv("./countywide_covid_19_metadata.csv")

In [20]:
mdf.head(n=10)

,Code,Variable,Description,Source
0,FIPS,FIPS,State-County FIPS Code,NaN
1,State,State,State Abbreviation,NaN
2,County,County,County Name,NaN
3,X01,ECONOMY_TYPE,County economic types: 0=Nonspecialized 1=Farm...,https://www.ers.usda.gov
4,X02,POPULATION,Estimated resident population in 2019,https://www.ers.usda.gov
5,X03,IMMIGRATION_RATE,Net international migration rate (migrants per...,https://www.ers.usda.gov
6,X04,RURAL_URBAN_CONTINUUM,A code indicating level of urbanization. 1 = m...,https://www.ers.usda.gov
7,X05,URBAN_INFLUENCE,A code indicating nearness of urban areas. 1 =...,https://www.ers.usda.gov
8,X06,PCT_NO_HS_DIPLOMA,Percent of population with no high school diploma,https://www.ers.usda.gov
9,X07,PCT_HS_DIPLOMA_ONLY,Percent of population with only a high school ...,https://www.ers.usda.gov


In [21]:
# get metadata descriptions
X_desc = mdf[mdf.Code.isin(X.columns.values)]
print("")
print("Selected features:")
print(X_desc.iloc[:,0:3])


Selected features:
   Code               Variable  \
3   X01           ECONOMY_TYPE   
4   X02             POPULATION   
5   X03       IMMIGRATION_RATE   
6   X04  RURAL_URBAN_CONTINUUM   
7   X05        URBAN_INFLUENCE   
8   X06      PCT_NO_HS_DIPLOMA   
9   X07    PCT_HS_DIPLOMA_ONLY   
10  X08       PCT_SOME_COLLEGE   
11  X09       PCT_BS_OR_HIGHER   
12  X10         PCT_IN_POVERTY   

                                          Description  
3   County economic types: 0=Nonspecialized 1=Farm...  
4               Estimated resident population in 2019  
5   Net international migration rate (migrants per...  
6   A code indicating level of urbanization. 1 = m...  
7   A code indicating nearness of urban areas. 1 =...  
8   Percent of population with no high school diploma  
9   Percent of population with only a high school ...  
10            Percent of population with some college  
11  Percent of population with bachelor's degree o...  
12   Percent of population living below pover

In [22]:
Y_desc = mdf[mdf.Code == label]
print("")
print("Selected label:")
print(Y_desc.iloc[:,0:3])


Selected label:
   Code Variable                                        Description
61  Y05      DRP  Death rate percentile (1 = 10th percentile; 2 ...


In [23]:
# encode labels per needs of RF algorithm
Y = pd.get_dummies(Y)

In [24]:
Y.head(n=10)

,1,2,3,4,5,6,7,8,9,10
0,0,0,1,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,1,0,0,0,0
8,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,0,1,0,0,0


In [25]:
#
# Part 3 - create and train the model
#

# split into train and test datasets
X_train, X_test, \
Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

In [26]:
# create and train the classifier
model = RandomForestClassifier(\
    criterion='gini',          \
    oob_score=True,            \
    random_state=1)

In [27]:
model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)

In [28]:
# feature importance
fimp = pd.DataFrame(model.feature_importances_, columns=["Importance"])
fimp["Feature"] = X.columns
fimp["Description"] = X_desc["Variable"].to_list()
fimp = fimp[["Feature", "Importance", "Description"]]
fimp = fimp.sort_values("Importance", ascending=False)
print("")
print("Feature  Importance  Description")
for index,item in fimp.iterrows():
    print("{:7s}  {:10.8f}  {:s}".format(item["Feature"],item["Importance"], item["Description"]))
print("")


Feature  Importance  Description
X02      0.13898782  POPULATION
X10      0.12588257  PCT_IN_POVERTY
X08      0.12393476  PCT_SOME_COLLEGE
X07      0.12295908  PCT_HS_DIPLOMA_ONLY
X09      0.12163032  PCT_BS_OR_HIGHER
X06      0.12145114  PCT_NO_HS_DIPLOMA
X03      0.09656596  IMMIGRATION_RATE
X05      0.05347359  URBAN_INFLUENCE
X01      0.05139690  ECONOMY_TYPE
X04      0.04371786  RURAL_URBAN_CONTINUUM



In [29]:
#
# Part 4 - evaluate the model using test data
#

n_test    = len(Y_test)
n_classes = Y.shape[1]

In [30]:
print(n_test)

785


In [31]:
print(n_classes)

10


In [32]:
probs = model.predict_proba(X_test)

In [33]:
print(probs)

[array([[0.96, 0.04],
       [0.96, 0.04],
       [0.98, 0.02],
       ...,
       [0.88, 0.12],
       [0.84, 0.16],
       [0.74, 0.26]]), array([[0.81, 0.19],
       [0.98, 0.02],
       [0.65, 0.35],
       ...,
       [0.71, 0.29],
       [0.91, 0.09],
       [0.98, 0.02]]), array([[0.87, 0.13],
       [0.94, 0.06],
       [0.93, 0.07],
       ...,
       [0.84, 0.16],
       [0.87, 0.13],
       [0.97, 0.03]]), array([[0.91, 0.09],
       [0.96, 0.04],
       [0.76, 0.24],
       ...,
       [0.95, 0.05],
       [0.71, 0.29],
       [0.96, 0.04]]), array([[0.91, 0.09],
       [0.97, 0.03],
       [0.92, 0.08],
       ...,
       [0.83, 0.17],
       [0.89, 0.11],
       [0.98, 0.02]]), array([[0.88, 0.12],
       [0.94, 0.06],
       [0.91, 0.09],
       ...,
       [0.94, 0.06],
       [0.93, 0.07],
       [0.99, 0.01]]), array([[0.92, 0.08],
       [0.96, 0.04],
       [0.93, 0.07],
       ...,
       [0.94, 0.06],
       [0.96, 0.04],
       [0.97, 0.03]]), array([[0.91, 0.09]

In [34]:
pi = [ [ probs[i][j][1] for i in range(0,n_classes)] for j in range(0,n_test)]

In [36]:
print(pi)

[[0.04, 0.19, 0.13, 0.09, 0.09, 0.12, 0.08, 0.09, 0.14, 0.03], [0.04, 0.02, 0.06, 0.04, 0.03, 0.06, 0.04, 0.1, 0.2, 0.41], [0.02, 0.35, 0.07, 0.24, 0.08, 0.09, 0.07, 0.03, 0.05, 0.0], [0.09, 0.16, 0.06, 0.07, 0.13, 0.12, 0.08, 0.15, 0.11, 0.03], [0.06, 0.08, 0.03, 0.09, 0.05, 0.06, 0.19, 0.05, 0.22, 0.17], [0.03, 0.15, 0.15, 0.11, 0.05, 0.16, 0.12, 0.1, 0.08, 0.05], [0.05, 0.02, 0.08, 0.06, 0.08, 0.19, 0.25, 0.1, 0.12, 0.05], [0.02, 0.05, 0.05, 0.08, 0.05, 0.1, 0.06, 0.12, 0.38, 0.09], [0.17, 0.05, 0.07, 0.04, 0.11, 0.11, 0.17, 0.07, 0.11, 0.1], [0.18, 0.3, 0.08, 0.07, 0.09, 0.03, 0.03, 0.07, 0.02, 0.13], [0.01, 0.12, 0.13, 0.2, 0.05, 0.02, 0.05, 0.2, 0.1, 0.12], [0.04, 0.06, 0.12, 0.07, 0.22, 0.11, 0.12, 0.09, 0.07, 0.1], [0.01, 0.08, 0.15, 0.12, 0.09, 0.12, 0.09, 0.23, 0.1, 0.01], [0.04, 0.29, 0.08, 0.03, 0.05, 0.09, 0.11, 0.05, 0.12, 0.14], [0.23, 0.18, 0.04, 0.1, 0.02, 0.04, 0.03, 0.05, 0.03, 0.28], [0.01, 0.01, 0.09, 0.09, 0.07, 0.1, 0.07, 0.29, 0.22, 0.05], [0.04, 0.05, 0.11, 0.1

In [37]:
Y_pred = [1+np.argmax(pi[i]) for i in range(0,n_test)]

In [38]:
print(Y_pred)

[2, 10, 2, 2, 9, 6, 7, 9, 1, 2, 4, 5, 8, 2, 10, 8, 4, 10, 5, 1, 1, 2, 6, 1, 9, 6, 1, 2, 7, 1, 8, 3, 10, 6, 7, 10, 5, 10, 9, 3, 3, 2, 6, 2, 9, 5, 3, 1, 1, 1, 1, 6, 9, 7, 8, 1, 7, 1, 8, 4, 3, 10, 7, 2, 6, 10, 10, 1, 8, 1, 6, 3, 9, 2, 6, 9, 1, 9, 2, 1, 9, 4, 10, 3, 8, 4, 1, 4, 5, 9, 1, 2, 1, 6, 7, 5, 1, 3, 9, 3, 2, 4, 7, 2, 8, 1, 10, 10, 2, 9, 2, 10, 3, 5, 10, 4, 5, 9, 7, 9, 3, 7, 8, 4, 3, 6, 4, 9, 10, 9, 8, 9, 7, 6, 1, 7, 5, 2, 1, 10, 9, 2, 8, 10, 3, 8, 7, 8, 1, 10, 1, 1, 5, 2, 6, 4, 2, 3, 9, 8, 6, 7, 6, 10, 10, 8, 8, 3, 6, 4, 10, 4, 5, 6, 7, 7, 4, 7, 3, 10, 6, 1, 6, 10, 9, 6, 6, 9, 7, 9, 3, 1, 9, 1, 7, 5, 2, 10, 8, 8, 5, 7, 1, 8, 6, 1, 1, 1, 3, 5, 8, 2, 8, 2, 6, 7, 1, 5, 1, 8, 2, 9, 5, 10, 1, 3, 9, 5, 4, 6, 2, 7, 10, 1, 5, 6, 10, 6, 5, 4, 10, 3, 9, 10, 3, 1, 8, 2, 2, 5, 5, 9, 4, 6, 4, 8, 10, 2, 9, 8, 2, 4, 4, 7, 4, 5, 10, 6, 10, 4, 3, 9, 6, 7, 3, 3, 2, 10, 5, 9, 8, 8, 4, 10, 4, 7, 10, 10, 7, 7, 9, 5, 6, 5, 3, 9, 4, 3, 4, 2, 2, 5, 3, 10, 6, 10, 6, 3, 5, 5, 10, 8, 2, 3, 5, 1, 9, 8, 7, 5, 

In [39]:
Y_actual = [1+np.argmax(Y_test.iloc[i,:]) for i in range(0,n_test)]


In [40]:
print(Y_actual)

[3, 10, 1, 8, 2, 2, 4, 8, 9, 2, 10, 1, 5, 8, 10, 7, 7, 5, 1, 9, 1, 3, 10, 4, 10, 3, 1, 5, 1, 10, 2, 2, 9, 7, 4, 7, 7, 10, 2, 3, 4, 4, 3, 7, 8, 8, 7, 5, 6, 2, 7, 5, 4, 3, 1, 2, 1, 2, 9, 9, 10, 8, 8, 2, 1, 9, 6, 3, 6, 1, 6, 6, 7, 6, 1, 6, 2, 9, 4, 1, 10, 3, 9, 2, 6, 5, 4, 7, 8, 4, 5, 6, 4, 5, 6, 2, 1, 3, 9, 1, 1, 10, 4, 6, 8, 6, 7, 9, 10, 7, 4, 9, 2, 5, 9, 6, 1, 5, 7, 4, 6, 8, 4, 1, 9, 3, 7, 4, 10, 8, 5, 2, 8, 10, 5, 9, 4, 2, 2, 5, 2, 2, 3, 7, 1, 8, 8, 7, 4, 9, 7, 4, 2, 1, 9, 5, 3, 4, 5, 5, 2, 3, 1, 2, 9, 9, 8, 9, 5, 9, 9, 7, 4, 3, 4, 9, 5, 2, 2, 8, 10, 10, 6, 10, 8, 6, 4, 8, 2, 10, 4, 10, 10, 1, 6, 7, 8, 10, 9, 9, 9, 2, 10, 10, 6, 1, 6, 5, 2, 10, 9, 3, 10, 1, 6, 3, 2, 3, 10, 5, 1, 8, 3, 4, 1, 5, 3, 5, 3, 8, 6, 1, 9, 8, 5, 6, 10, 1, 3, 5, 2, 9, 3, 4, 1, 8, 8, 9, 1, 7, 9, 5, 7, 6, 2, 10, 8, 7, 6, 6, 3, 2, 2, 4, 5, 6, 8, 6, 8, 3, 3, 5, 3, 4, 6, 3, 1, 7, 3, 5, 9, 8, 7, 9, 4, 5, 2, 10, 4, 6, 2, 5, 8, 6, 5, 7, 6, 2, 6, 4, 8, 6, 5, 10, 9, 2, 9, 7, 7, 9, 3, 8, 8, 2, 7, 3, 2, 10, 6, 5, 9, 9, 9, 

In [41]:
ca = accuracy_score(Y_actual, Y_pred)

In [42]:
print("Classification Accuracy : {:.2f}".format(ca))

Classification Accuracy : 0.17


In [43]:
cm = confusion_matrix(Y_actual, Y_pred)

In [44]:
C = set(list(Y_pred)+list(Y_actual)) 

In [45]:
cm = pd.DataFrame(cm,index=C,columns=C)

In [46]:
print("")
print("Confusion Matrix")
print(cm)
print("")


Confusion Matrix
    1   2   3   4   5   6   7   8   9   10
1   30  13   3   5   3   6   5   1   1   9
2   14  15  11   9   5   5   5   1   9   8
3    8  15  12  12  10   9   6   7   5   3
4   11  11  11   5   9   7  15   2   6   5
5   12   6   8   9  13   3   7   6   6   3
6    5   6   8   5  11  11   8   7   6   2
7    4   6   3  10  17   2   7   9   7   9
8    6   8   3   4  11   8  10  11  15  15
9    4   1   6   7   7  10   7  12   8  21
10  13   2   1   4   6   4   3   9   5  21



In [48]:
# Let's re-do the anlaysis using SVM

#
# Part 1 - load the data and required modules
#
import sys
from sklearn import svm
# bring in normalization routine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve
import pandas as pd
import numpy as np
from six import StringIO
from matplotlib import pyplot as plt
import os